# Genie Bot - Development Notebook

Notebook para desenvolvimento e teste do Genie Bot linha a linha

In [ ]:
%pip install -U -qqqq databricks-openai backoff databricks-agents mlflow-skinny[databricks] unitycatalog-ai
dbutils.library.restartPython()

import json
import requests
from typing import Any, Callable, Generator
from uuid import uuid4

import mlflow
from databricks.sdk import WorkspaceClient
from mlflow.entities import SpanType
from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import (
    ResponsesAgentRequest,
    ResponsesAgentResponse,
    ResponsesAgentStreamEvent,
    output_to_responses_items_stream,
    to_chat_completions_input,
)
from pydantic import BaseModel

## Imports

In [ ]:
# Configurações
LLM_ENDPOINT_NAME = "databricks-llama-4-maverick"
GENIE_SPACE_ID = "SEU_SPACE_ID_AQUI"  # ← Substituir pelo seu Space ID

SYSTEM_PROMPT = """Você é um assistente que responde perguntas sobre dados CRM usando o Genie Space.
Para qualquer pergunta sobre dados, use a ferramenta query_genie para obter a resposta."""

## Configurações

**IMPORTANTE**: Edite o GENIE_SPACE_ID abaixo com o ID do seu Genie Space

In [ ]:
# Configurações
LLM_ENDPOINT_NAME = "databricks-llama-4-maverick"
GENIE_SPACE_ID = "SEU_SPACE_ID_AQUI"  # ← Substituir pelo seu Space ID

SYSTEM_PROMPT = """Você é um assistente que responde perguntas sobre dados CRM usando o Genie Space.
Para qualquer pergunta sobre dados, use a ferramenta query_genie para obter a resposta."""

# Definir a tool query_genie como função Python nativa
def query_genie_tool(question: str) -> str:
    """
    Query the Genie Space with a natural language question about CRM data.

    Args:
        question: Natural language question to ask Genie

    Returns:
        Answer from Genie Space with data and insights
    """
    try:
        # Obter workspace client (já tem autenticação)
        workspace_client = WorkspaceClient()

        # Pegar host e token do workspace client
        api_client = workspace_client.api_client
        host = api_client.host
        token = api_client.token

        # API endpoint
        url = f"{host}/api/2.0/genie/spaces/{GENIE_SPACE_ID}/start-conversation"

        headers = {
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json"
        }

        payload = {"content": question}

        response = requests.post(url, headers=headers, json=payload, timeout=60)
        response.raise_for_status()
        result = response.json()

        # Extrair resposta do Genie
        if isinstance(result, dict):
            # Tentar pegar o conteúdo da mensagem
            if "message" in result and isinstance(result["message"], dict):
                if "content" in result["message"]:
                    return result["message"]["content"]

            # Tentar pegar attachments (query results)
            if "attachments" in result:
                attachments = result["attachments"]
                if attachments and len(attachments) > 0:
                    first_attachment = attachments[0]
                    if "text" in first_attachment:
                        return first_attachment["text"]["content"]
                    if "query" in first_attachment:
                        query_result = first_attachment["query"]
                        if "result" in query_result:
                            return json.dumps(query_result["result"], ensure_ascii=False, indent=2)

            # Se não encontrou, retornar o JSON completo
            return json.dumps(result, ensure_ascii=False, indent=2)

        return str(result)

    except requests.exceptions.Timeout:
        return "Error: Genie query timed out after 60 seconds. Try a simpler question."
    except requests.exceptions.RequestException as e:
        return f"Error calling Genie API: {str(e)}"
    except Exception as e:
        return f"Error: {str(e)}"

print("query_genie_tool function defined successfully")

In [ ]:
## Create Tool Specs

**Não precisa mais de UC Function!** A tool agora é uma função Python nativa.

In [ ]:
# Criar tool spec manualmente
TOOL_INFOS = [
    ToolInfo(
        name="query_genie",
        spec={
            "type": "function",
            "function": {
                "name": "query_genie",
                "description": "Query the Genie Space with a natural language question about CRM data. Use this for any questions about verifications, risks, deviations, users, actions, or any CRM metrics.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "Natural language question to ask Genie about CRM data"
                        }
                    },
                    "required": ["question"]
                }
            }
        },
        exec_fn=query_genie_tool
    )
]

print(f"Loaded {len(TOOL_INFOS)} tools:")
for tool in TOOL_INFOS:
    print(f"  - {tool.name}")

## Load UC Function Tools

**Pré-requisito**: A UC Function `hs_franquia.gold_connect_bot.query_genie` deve estar criada

In [ ]:
# UC Function: query_genie
UC_TOOL_NAMES = ["hs_franquia.gold_connect_bot.query_genie"]
uc_toolkit = UCFunctionToolkit(function_names=UC_TOOL_NAMES)
TOOL_INFOS = [create_tool_info(tool_spec) for tool_spec in uc_toolkit.tools]

print(f"Loaded {len(TOOL_INFOS)} tools:")
for tool in TOOL_INFOS:
    print(f"  - {tool.name}")

## GenieBot Class Definition

In [ ]:
class GenieBot(ResponsesAgent):
    def __init__(self, llm_endpoint: str, tools: list[ToolInfo]):
        self.llm_endpoint = llm_endpoint
        self.workspace_client = WorkspaceClient()
        self.model_serving_client = self.workspace_client.serving_endpoints.get_open_ai_client()
        self._tools_dict = {tool.name: tool for tool in tools}

    def get_tool_specs(self) -> list[dict]:
        return [tool_info.spec for tool_info in self._tools_dict.values()]

    @mlflow.trace(span_type=SpanType.TOOL)
    def execute_tool(self, tool_name: str, args: dict) -> Any:
        return self._tools_dict[tool_name].exec_fn(**args)

    def call_llm(self, messages: list[dict[str, Any]]) -> Generator[dict[str, Any], None, None]:
        for chunk in self.model_serving_client.chat.completions.create(
            model=self.llm_endpoint,
            messages=to_chat_completions_input(messages),
            tools=self.get_tool_specs(),
            stream=True,
        ):
            chunk_dict = chunk.to_dict()
            if len(chunk_dict.get("choices", [])) > 0:
                yield chunk_dict

    def handle_tool_call(self, tool_call: dict[str, Any], messages: list[dict[str, Any]]) -> ResponsesAgentStreamEvent:
        args = json.loads(tool_call["arguments"])
        result = str(self.execute_tool(tool_name=tool_call["name"], args=args))

        tool_call_output = self.create_function_call_output_item(tool_call["call_id"], result)
        messages.append(tool_call_output)
        return ResponsesAgentStreamEvent(type="response.output_item.done", item=tool_call_output)

    def call_and_run_tools(self, messages: list[dict[str, Any]], max_iter: int = 10) -> Generator[ResponsesAgentStreamEvent, None, None]:
        for _ in range(max_iter):
            last_msg = messages[-1]
            if last_msg.get("role", None) == "assistant":
                return
            elif last_msg.get("type", None) == "function_call":
                yield self.handle_tool_call(last_msg, messages)
            else:
                yield from output_to_responses_items_stream(
                    chunks=self.call_llm(messages), aggregator=messages
                )

        yield ResponsesAgentStreamEvent(
            type="response.output_item.done",
            item=self.create_text_output_item("Max iterations reached.", str(uuid4())),
        )

    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        outputs = [
            event.item
            for event in self.predict_stream(request)
            if event.type == "response.output_item.done"
        ]
        return ResponsesAgentResponse(output=outputs, custom_outputs=request.custom_inputs)

    def predict_stream(self, request: ResponsesAgentRequest) -> Generator[ResponsesAgentStreamEvent, None, None]:
        messages = to_chat_completions_input([i.model_dump() for i in request.input])
        if SYSTEM_PROMPT:
            messages.insert(0, {"role": "system", "content": SYSTEM_PROMPT})
        yield from self.call_and_run_tools(messages=messages)

print("GenieBot class defined successfully")

## Initialize Agent

In [ ]:
# Inicializar agent
mlflow.openai.autolog()
AGENT = GenieBot(llm_endpoint=LLM_ENDPOINT_NAME, tools=TOOL_INFOS)

print(f"Agent initialized with:")
print(f"  - LLM endpoint: {LLM_ENDPOINT_NAME}")
print(f"  - Tools: {list(AGENT._tools_dict.keys())}")

## Test: Simple Query

In [ ]:
# Teste simples
test_request = {
    "input": [{
        "role": "user",
        "content": "Quantas verificações tivemos em 2024?"
    }]
}

result = AGENT.predict(test_request)
print("Response:")
print(result)

## Test: Extract Text from Response

In [ ]:
# Extrair texto da resposta
if result and hasattr(result, 'output') and len(result.output) > 0:
    last_output = result.output[-1]
    if hasattr(last_output, 'content'):
        print("\nResposta formatada:")
        print(last_output.content)
    else:
        print("\nOutput completo:")
        print(last_output)

## Test: Streaming Response

In [ ]:
# Teste com streaming
print("Testing streaming response...\n")

for event in AGENT.predict_stream(test_request):
    print(f"Event type: {event.type}")
    if event.type == "response.output_item.done":
        print(f"Content: {event.item}")
    print("---")

## Test: Multiple Queries

In [ ]:
# Teste com múltiplas queries
test_queries = [
    "Quantas verificações tivemos em 2024?",
    "Quais são os top 10 riscos com mais desvios?",
    "Me mostre os usuários com mais ações em aberto"
]

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*60}")
    print(f"Query {i}: {query}")
    print('='*60)
    
    result = AGENT.predict({
        "input": [{"role": "user", "content": query}]
    })
    
    if result and hasattr(result, 'output') and len(result.output) > 0:
        last_output = result.output[-1]
        if hasattr(last_output, 'content'):
            print(last_output.content)
        else:
            print(last_output)

## Test: Conversation Context

In [ ]:
# Teste com contexto de conversa
conversation = {
    "input": [
        {
            "role": "user",
            "content": "Quantas verificações em 2024?"
        },
        {
            "role": "assistant",
            "content": "Foram realizadas 3877 verificações em 2024."
        },
        {
            "role": "user",
            "content": "E em 2023?"
        }
    ]
}

print("Testing conversation with context...\n")
result = AGENT.predict(conversation)

if result and hasattr(result, 'output') and len(result.output) > 0:
    last_output = result.output[-1]
    if hasattr(last_output, 'content'):
        print(last_output.content)
    else:
        print(last_output)

## Debug: Test Genie Tool Directly

In [ ]:
# Debug: Testar query_genie_tool diretamente
result = query_genie_tool("Quantas verificações em 2024?")

print("Genie tool result:")
print(result)

## Debug: Test UC Function Directly

In [ ]:
# Debug: Testar UC Function diretamente
from unitycatalog.ai.core.base import get_uc_function_client

uc_client = get_uc_function_client()
result = uc_client.execute_function(
    "hs_franquia.gold_connect_bot.query_genie",
    {
        "question": "Quantas verificações em 2024?",
        "space_id": GENIE_SPACE_ID
    }
)

print("UC Function result:")
if result.error:
    print(f"Error: {result.error}")
else:
    print(f"Value: {result.value}")

## Set Model for MLflow

Execute esta célula quando estiver pronto para fazer o deploy

In [ ]:
# Registrar o modelo para MLflow
mlflow.models.set_model(AGENT)
print("Agent registered with MLflow")

## Next Steps

Após validar que tudo funciona:
1. Execute as células acima em ordem
2. Valide que os testes retornam respostas corretas do Genie
3. Use `deploy_genie_bot.ipynb` para fazer o deployment completo

Ou copie o conteúdo deste notebook para `genie_bot.py` usando `%%writefile` se preferir deploy via arquivo Python.